# Northwind database
##### Jonathan Tyler

Northwind is a company that sells specialty foods. I was given a mandate to  "__do something with this database__" as my first project as a new hire. Unfortunatly at the end of last week, the whole computer science division got sick. It was someone's birthday and they all ate bad sheet cake. I wasn't aware those could *actually* go bad.

I was able to find a entity-relation diagram for the database but not much else. So to break down this task:
1. First I am going to have to explore the database myself to see the basic metrics of the company.
2. Then I will use this basic data to formulate some hypotheses concerning some underlying trends.
3. Finally I will attempt to test out these ideas to prove myself right or wrong.

## EDA

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session, sessionmaker

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from scipy.stats import ttest_1samp, ttest_ind
from scipy.stats import levene, shapiro, mannwhitneyu, kruskal

plt.style.use('ggplot')

Connecting the database enable to start looking at the data. I plan to compare the ERD to the database and get basic metric from the tables.

In [5]:
#engine = create_engine('sqlite:///Northwind_large.sqlite', echo=True)
engine = create_engine('sqlite:///Northwind_small.sqlite', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

inspector = inspect(engine) #checking the ERD against the actual database
inspector.get_table_names()

2019-02-01 13:04:00,203 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-01 13:04:00,205 INFO sqlalchemy.engine.base.Engine ()
2019-02-01 13:04:00,207 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-01 13:04:00,208 INFO sqlalchemy.engine.base.Engine ()
2019-02-01 13:04:00,210 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-02-01 13:04:00,210 INFO sqlalchemy.engine.base.Engine ()


['Category',
 'Customer',
 'CustomerCustomerDemo',
 'CustomerDemographic',
 'Employee',
 'EmployeeTerritory',
 'Order',
 'OrderDetail',
 'Product',
 'Region',
 'Shipper',
 'Supplier',
 'Territory']

So found my first discrepancy, most the table names are stated as singular while the ERD have them as plural.  
  
I will submit a ticket to the department to fix this when they get back.

In [6]:
inspector.get_columns('Product')

2019-02-01 13:04:00,221 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Product")
2019-02-01 13:04:00,222 INFO sqlalchemy.engine.base.Engine ()


[{'name': 'Id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'ProductName',
  'type': VARCHAR(length=8000),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'SupplierId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'CategoryId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'QuantityPerUnit',
  'type': VARCHAR(length=8000),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'UnitPrice',
  'type': DECIMAL(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'UnitsInStock',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'UnitsOnOrder',
  'type': INTEGER(),
  'n

A bit messy, I recall a function that I had used previously that I could use to help in cleaning this up.

In [7]:
def get_column_info(col_name): #quick function based on https://github.com/learn-co-curriculum/dsc-2-13-11-queries-with-sqlalchemy-lab
    col_list = inspector.get_columns(col_name)
    print(f'Table Name: {col_name} \n')
    
    for col in col_list:
        if col['primary_key'] == 1:
            print(f"{col['name']}  ||PRIMARY KEY||  dtype: {col['type']}")
        else:
            print(f"{col['name']}     dtype: {col['type']}")
                  

In [8]:
get_column_info('Product')

Table Name: Product 

Id  ||PRIMARY KEY||  dtype: INTEGER
ProductName     dtype: VARCHAR(8000)
SupplierId     dtype: INTEGER
CategoryId     dtype: INTEGER
QuantityPerUnit     dtype: VARCHAR(8000)
UnitPrice     dtype: DECIMAL
UnitsInStock     dtype: INTEGER
UnitsOnOrder     dtype: INTEGER
ReorderLevel     dtype: INTEGER
Discontinued     dtype: INTEGER


In [9]:
get_column_info('Employee')

2019-02-01 13:04:00,355 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Employee")
2019-02-01 13:04:00,358 INFO sqlalchemy.engine.base.Engine ()
Table Name: Employee 

Id  ||PRIMARY KEY||  dtype: INTEGER
LastName     dtype: VARCHAR(8000)
FirstName     dtype: VARCHAR(8000)
Title     dtype: VARCHAR(8000)
TitleOfCourtesy     dtype: VARCHAR(8000)
BirthDate     dtype: VARCHAR(8000)
HireDate     dtype: VARCHAR(8000)
Address     dtype: VARCHAR(8000)
City     dtype: VARCHAR(8000)
Region     dtype: VARCHAR(8000)
PostalCode     dtype: VARCHAR(8000)
Country     dtype: VARCHAR(8000)
HomePhone     dtype: VARCHAR(8000)
Extension     dtype: VARCHAR(8000)
Photo     dtype: BLOB
Notes     dtype: VARCHAR(8000)
ReportsTo     dtype: INTEGER
PhotoPath     dtype: VARCHAR(8000)


In [10]:
get_column_info('Supplier')

2019-02-01 13:04:00,380 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Supplier")
2019-02-01 13:04:00,381 INFO sqlalchemy.engine.base.Engine ()
Table Name: Supplier 

Id  ||PRIMARY KEY||  dtype: INTEGER
CompanyName     dtype: VARCHAR(8000)
ContactName     dtype: VARCHAR(8000)
ContactTitle     dtype: VARCHAR(8000)
Address     dtype: VARCHAR(8000)
City     dtype: VARCHAR(8000)
Region     dtype: VARCHAR(8000)
PostalCode     dtype: VARCHAR(8000)
Country     dtype: VARCHAR(8000)
Phone     dtype: VARCHAR(8000)
Fax     dtype: VARCHAR(8000)
HomePage     dtype: VARCHAR(8000)


  
So a quick check of the different tables align with the ERD. That is good. But I should be checking out the individual tables before calling them nevertheless, it is good pratice.
  
Now lets start checking out a few things:
- How much of what are we selling?
- Who are our main suppliers?
- What does our customer base look like?
- What is the geographical spread of our workforce?

Once we know these things, we will have a broad overview of the business. From there we will investigate any abnormalities or go splunking for underlying trends.
___
---
Now lets make a connection to the engine and make sure it works.

In [11]:
con = engine.connect() #connecting the engine to be able to make queries

In [12]:
q = '''SELECT * FROM [Order]''' #simple query
df_order = pd.read_sql_query(q, engine) #puts the information from the query into a dataframe
df_order.head()

2019-02-01 13:04:00,459 INFO sqlalchemy.engine.base.Engine SELECT * FROM [Order]
2019-02-01 13:04:00,461 INFO sqlalchemy.engine.base.Engine ()


,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,VICTE,3,2012-07-08,2012-08-05,2012-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium


Fantasic, now lets start flushing out answers to those inital questions

## How much of what are we selling?

Lets check the Product table and the Order Detail table

In [13]:
print(get_column_info('Product'))
print(get_column_info('OrderDetail'))
print(get_column_info('Category'))

Table Name: Product 

Id  ||PRIMARY KEY||  dtype: INTEGER
ProductName     dtype: VARCHAR(8000)
SupplierId     dtype: INTEGER
CategoryId     dtype: INTEGER
QuantityPerUnit     dtype: VARCHAR(8000)
UnitPrice     dtype: DECIMAL
UnitsInStock     dtype: INTEGER
UnitsOnOrder     dtype: INTEGER
ReorderLevel     dtype: INTEGER
Discontinued     dtype: INTEGER
None
2019-02-01 13:04:00,564 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("OrderDetail")
2019-02-01 13:04:00,566 INFO sqlalchemy.engine.base.Engine ()
Table Name: OrderDetail 

Id  ||PRIMARY KEY||  dtype: VARCHAR(8000)
OrderId     dtype: INTEGER
ProductId     dtype: INTEGER
UnitPrice     dtype: DECIMAL
Quantity     dtype: INTEGER
Discount     dtype: FLOAT
None
2019-02-01 13:04:00,570 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Category")
2019-02-01 13:04:00,571 INFO sqlalchemy.engine.base.Engine ()
Table Name: Category 

Id  ||PRIMARY KEY||  dtype: INTEGER
CategoryName     dtype: VARCHAR(8000)
Description     dtype: VARCH

Looks like the table is ERD is wrong again. Some of the tables are incorrectly stated i.e. ProductID is just Id

In [14]:
q='''SELECT p.ProductName, c.CategoryName, SUM(o.Quantity) num_ordered \
FROM Product p \
LEFT JOIN OrderDetail o ON o.ProductId = p.Id \
LEFT JOIN Category c ON c.Id = p.CategoryId \
GROUP BY p.ProductName ORDER BY num_ordered DESC'''
df1 = pd.read_sql_query(q, engine)
df1.head()

2019-02-01 13:04:00,587 INFO sqlalchemy.engine.base.Engine SELECT p.ProductName, c.CategoryName, SUM(o.Quantity) num_ordered FROM Product p LEFT JOIN OrderDetail o ON o.ProductId = p.Id LEFT JOIN Category c ON c.Id = p.CategoryId GROUP BY p.ProductName ORDER BY num_ordered DESC
2019-02-01 13:04:00,588 INFO sqlalchemy.engine.base.Engine ()


,ProductName,CategoryName,num_ordered
0,Camembert Pierrot,Dairy Products,1577
1,Raclette Courdavault,Dairy Products,1496
2,Gorgonzola Telino,Dairy Products,1397
3,Gnocchi di nonna Alice,Grains/Cereals,1263
4,Pavlova,Confections,1158


In [15]:
df1.CategoryName.value_counts()

Confections       13
Condiments        12
Seafood           12
Beverages         12
Dairy Products    10
Grains/Cereals     7
Meat/Poultry       6
Produce            5
Name: CategoryName, dtype: int64

In [20]:
q='''SELECT * FROM OrderDetail'''
df_od = pd.read_sql_query(q, engine)
df_od.head()

2019-02-01 13:04:35,667 INFO sqlalchemy.engine.base.Engine SELECT * FROM OrderDetail
2019-02-01 13:04:35,668 INFO sqlalchemy.engine.base.Engine ()


,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [19]:
q='''SELECT * FROM Product'''
df_prod = pd.read_sql_query(q, engine)
df_prod.head()

2019-02-01 13:04:34,050 INFO sqlalchemy.engine.base.Engine SELECT * FROM Product
2019-02-01 13:04:34,050 INFO sqlalchemy.engine.base.Engine ()


,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [21]:
df_prod.set_index('Id',inplace=True)
df_od.set_index('ProductId',inplace=True)

In [26]:
df_prod = df_prod.sort_index()
df_od = df_od.sort_index()

In [28]:
df_od.UnitPrice - df_prod.UnitPrice

1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1    -3.6
1     0.0
1     0.0
1     0.0
1     0.0
1    -3.6
1    -3.6
1    -3.6
1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1    -3.6
1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1     0.0
1    -3.6
1     0.0
     ... 
77   -2.6
77   -2.6
77    0.0
77    0.0
77   -2.6
77   -2.6
77   -2.6
77   -2.6
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77    0.0
77   -2.6
77   -2.6
77    0.0
77    0.0
Name: UnitPrice, Length: 2155, dtype: float64

This dataset contains order details on 2155 items ordered by different companies. Also looks like confections is the kind of food we have the most orders for.

## Who are our main suppliers?

In [ ]:
print(get_column_info('Order'))
print(get_column_info('Supplier'))

In [ ]:
q = '''SELECT s.CompanyName, s.Region, COUNT(*) num_of_orders FROM [Order] o \
LEFT JOIN OrderDetail od ON o.Id = od.OrderId \
LEFT JOIN Product p ON od.ProductId = p.Id \
LEFT JOIN Supplier s ON p.SupplierId = s.Id \
GROUP BY s.CompanyName \
ORDER BY num_of_orders DESC'''

df2 = pd.read_sql_query(q, engine)
df2.head()

In [ ]:
df2.Region.value_counts(), df2.num_of_orders.mean()

The largest amount of orders come from Western Europe and it is one of the largest portion of our supply chain.

## What does our customer base look like?

In [ ]:
print(get_column_info('CustomerDemographic'))
print(get_column_info('Customer'))
print(get_column_info('CustomerCustomerDemo'))

In [ ]:
q = '''SELECT * FROM CustomerCustomerDemo'''

df3 = pd.read_sql_query(q, engine)
df3

It looks like `CustomerCustomerDemo` is an empty table. This must be a new table or something went wrong. That means the only customer data I have to look at will from the customer table. Another ticket I need to submit.

In [ ]:
q = '''SELECT ContactTitle, Count(*) num_of_types \
FROM Customer  \
GROUP BY ContactTitle  \
ORDER BY num_of_types DESC'''

df4 = pd.read_sql_query(q, engine)
df4

In [ ]:
q = '''SELECT Region, Count(*) num_of_customers \
FROM Customer  \
GROUP BY Region  \
ORDER BY num_of_customers DESC'''

df5 = pd.read_sql_query(q, engine)
df5

It is good to see our supplier network lines up with our supply chain. Also intriguing to see what are the title of our contacts. Now finally to answer our last question.

## What is the geographical spread of our workforce?

In [2]:
print(get_column_info('Territory'))
print(get_column_info('Region'))
print(get_column_info('EmployeeTerritory'))
print(get_column_info('Employee'))

NameError: name 'get_column_info' is not defined

In [ ]:
q = '''SELECT e.LastName, e.Title, e.Region as based_from, r.RegionDescription FROM Employee e \
LEFT JOIN EmployeeTerritory et ON e.Id = et.EmployeeId \
LEFT JOIN Territory t ON et.TerritoryId = t.Id \
LEFT JOIN Region r ON t.RegionId = r.Id \
GROUP BY e.LastName \
ORDER BY e.Title'''

df6 = pd.read_sql_query(q, engine)
df6

It is unclear what "RegionDescription" from the `Region` table represents. I will need to do talk to one of the database engineers when they get back for some clarification.

So now we have a brief overview of the company, we can start some hypothesis testing.
___
___
# Hypothesis Testing
1) Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?  
2) Is there a difference in the money generated from sales between the North America office and the British Island office?  
3) Does having 'manager' in your title effect the amount a customer orders?  
4) Is there a difference in quantity of products given their average shelf life?
___
___

## 1) Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?

### 1-1 Initial Analysis
This was the one objective that I was given to solve using this database. I will gather the population metrics ($\mu, \sigma$) form the total number of products ordered. Then I will break up the orders into two groups (items with discount and items without a discount) and compare the two groups. For the hypothesis testing, I plan on using a one tailed test. People who plan on ordering a product won't order a smaller quantity solely because it cost less; therefore, there is little logic in using a two tailed test. 
  
If there is a significant result, I will further test out if there is a level of discount that sells more products.  

$H_{o}$: There is no difference in the number of products customers order given there is a discount  
$H_{a}$: There is an increase in the number of products customers order given there is a discount  
$\alpha = .05$

In [ ]:
q = '''SELECT OrderId, ProductId, Discount FROM OrderDetail'''

df = pd.read_sql_query(q, engine)
df.head(15)#, df.tail(15)

It looks like when Northwind Traders gives a discount to an order, it doesn't always apply across the whole order. Nor does the same discount level apply across the whole order. That means I have to reorganize my data.

In [ ]:
df.Discount.unique()

Northwind provides discount at a lot of different tiers. Grouping by order, I will take the largest amount of discount given. I am hoping this helps me bin the data into manageable subsections. Lets see how it looks.

In [ ]:
order_nums = df.OrderId.unique() #Gives the unique order number


#The below uses dictionary comprehension to create a dictionary that I will transform into a dataframe
#I matched the order number to orginal df. It counts how many rows/products made in that order.
#Then I found the max value of the discount given product-wise
dict_for_df = {num : [len(df[df.OrderId == num]), df.Discount[df.OrderId == num].max()] for num in order_nums}

df_order = pd.DataFrame.from_dict(dict_for_df, orient='index') #orient=index makes the keys of the dict be the index

df_order.head()

In [ ]:
df_order.reset_index(inplace=True) #reset the index, but keeping the orginal one in a new column
df_order.columns = ['id', 'num_of_items', 'lvl_of_disc'] #renaming to what they are
df_order.head()

In [ ]:
df_order.shape

Now a check to make sure I got everything from the orginal table.

In [ ]:
sum(df_order.num_of_items) == len(df)

Now lets take a look at what were the levels of discount offered.

In [ ]:
df_order.lvl_of_disc.value_counts()

This worked better than I could have hoped. The orders that were given small discounts (1% - 4%) to individual items, all had at least one item that was offered at 5% discount.  


In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='num_of_items',x='lvl_of_disc', data=df_order)
plt.title('Violin plots: comparing the shape at different levels of discount')
plt.xlabel('Max discount given per order')
plt.ylabel('Number of items in order');

So there seems to be an outlier at 20% discount. Lets take care of that and move onto testing.

In [ ]:
df_order.num_of_items[df_order.lvl_of_disc == .2].max() #finding the max at discount level .2

In [ ]:
df_order[df_order.num_of_items == 25].index

In [ ]:
df_order.drop(index=829, inplace=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='num_of_items',x='lvl_of_disc', data=df_order)
plt.title('Violin plots: comparing the shape at different levels of discount')
plt.xlabel('Max discount given per order')
plt.ylabel('Number of items in order')
#plt.savefig('violin discount levels');

### 1-2 Testing

Since there are multiple groups (levels of discount), an ANOVA test would be the best choice to use. The ANOVA test will show if there is a significant difference between no discount and discount. If there is a significant outcome, I plan on using the Tukey HSD (honestly significant difference) test to determine if the relationship between multiple levels of discount are statistically significant. This will be a post hoc test.


#### ANOVA
 
Checking the assumptions for an ANOVA testing. The assumptions apply to the residuals and not the variables themselves.
- Independence of cases.
- Normality of residuals
- Equality (or "homogeneity") of variances

Independence is met. Different orders from different customers meeting different demands. An ANOVA test is based on the F-statistic which is pretty robust and allows us to suspend some of the assumptions given the group sizes are roughly equal. We can verify the assumptions after the test by using the Levene’s or Omnibus test for homogeneity of variances and either the Jarque-Bera or Shapiro tests for Normality. More on this can be found _[here](https://pythonfordatascience.org/anova-python/)_.

Reformatting my hypothesis:  
$H_{o} : \mu_{no \space discount} = \mu_{discount}$  
$H_{a} : \mu_{no \space discount} < \mu_{discount}$  
$\alpha = 0.05$

In [ ]:
results = ols('num_of_items ~ C(lvl_of_disc)', data=df_order).fit()
results.summary()

Neither our Omnibus (homogenity) or Jarque-Bera (normailty) were signifigant meaning our assumptions for our test panned out. 

So focusing p-value of the F-statisic, it is less than our critical value. So there is a difference between the two groups and that means we have to dig a bit deeper. Finally to check at what level we see the biggest difference:

In [ ]:
mc = MultiComparison(df_order.num_of_items, df_order.lvl_of_disc)
mc_results = mc.tukeyhsd()
print(mc_results)

### 1-3 Results

- We can reject our null hypothesis that concludes that there is __significant difference__ between discount and no discount (p-value < $\alpha$ :: 0.0106 < 0.05)  
- Comparing between the different levels of discount, the greatest effect is seen at no discount to a 10% discount. 
- Further testing needs to account for orders with different discount levels within the same order
___
___

# 2) Is there a difference in the money generated from sales between the North America office and the British Island office?

The employees of this company are based in two central locations. I want to know if one office is better at business than the other. The metric will be money generated (products ordered * price) with the differential based on one office or another.

$H_{o} =$ The employees based out of the British Island office generates the same amount revenue, per order, than the North American office.  
$H_{a} =$ The employees based out of the British Island office generates less revenue, per order, than the North American office.  
$\alpha = 0.05$

### 2-1 Inital Analysis

During my EDA, the information for employee territory not specific for my needs. I will use the region they live in as the seperation.

In [ ]:
q = '''SELECT od.OrderId, SUM(od.UnitPrice*od.Quantity) AS generated, e.Region AS employ_from \
FROM OrderDetail od \
LEFT JOIN [Order] o ON od.OrderId = o.Id \
LEFT JOIN Employee e ON o.EmployeeId = e.Id
GROUP BY od.OrderId'''

df = pd.read_sql_query(q, engine)
df.head(10)

In [ ]:
df.shape

Looks like we have all 830 orders. Great.

In [ ]:
df.groupby('employ_from')['generated'].describe()

The average made by both offices are close to each other but there is far fewer sales made by the British office. Lets take a quick look at the distribution for both, but look at the above data there is likely to be outliers.

In [ ]:
plt.hist('generated', data=df[df.employ_from == 'North America'], label= 'NA')
plt.hist('generated', data=df[df.employ_from == 'British Isles'], label= 'BI')
plt.legend()
plt.ylabel('Quantities of order')
plt.xlabel('Price of order')
#plt.savefig('hist money')

There seems to be a few outliers. Lets take care of the egregious ones i.e. anything above 11K

In [ ]:
df.generated.sort_values(ascending=False).head(10)

In [ ]:
to_drop = [617,782,733,124,176,569,641,169]
df.drop(index=to_drop,inplace=True)

In [ ]:
brit = df[df.employ_from == 'British Isles']
merica = df[df.employ_from == 'North America']
brit.describe(), merica.describe()

## 2-2 Testing

### Mann–Whitney U test

Mann–Whitney U test is a nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample.

Unlike the t-test it does not require the assumption of normal distributions. It is nearly as efficient as the t-test on normal distributions.

Mann-Whitney assumptions:  
1) The dependent variable should be measured on an ordinal scale or a continuous scale.  
2) The independent variable should be two independent, categorical groups.  
3) Observations should be independent. In other words, there should be no relationship between the two groups or within each group.  
4) Observations are not normally distributed. However, they should follow the same shape (i.e. both are bell-shaped and skewed left).  


All assumptions are met within our data. So finally, refining our hypothesis: 

$H_{o} =$ There is no statistical difference between the meadian revenues generated by the North American office and the British Island office.  
$H_{a} =$ There is a statistical difference between the meadian revenues generated by the North American office and the British Island office.  
$\alpha = 0.05$

In [ ]:
merica['generated'].hist(label='American office')
brit['generated'].hist(label='British Isle office')
plt.title('Histogram of money generate by based on office location')
plt.xlabel('Quantity of money')
plt.ylabel('Orders count')
plt.legend()
plt.show()
#plt.savefig('hist of money gen')

In [ ]:
mannwhitneyu(brit.generated, merica.generated, alternative='two-sided')

#### Independent T-test

Like every test, this inferential statistic test has assumptions. The assumptions that the data must meet in order for the test results to be valid are:  

Independent T-test Assumptions
- The samples are independently and randomly drawn
- The distribution of the residuals between the two groups should follow the normal distribution
- The variances between the two groups are equal

The first assumption is met. The second assumption can be suspended given a large enough sample size, therefore independent t-test remains quite robust for violations of normality. [More information](http://thestatsgeek.com/2013/09/28/the-t-test-and-robustness-to-non-normality/) on this topic has been provided.

Keep in mind that as a parametric test, the independent t-test delivers best and most reliable results if both groups are normally distributed. Reliability decreases for skewed distributions.

In [3]:
ttest_ind(brit.generated, merica.generated)

NameError: name 'ttest_ind' is not defined

### 2-3 Results

- Failed to reject the null hypothesis. There is no statistical difference between the median revenues generated by the North American office and the British Island office 
- (p-value > $\alpha$ :: 0.3749 > 0.05)  
- Further testing: choosing a different metric and test out hypothesis again.
___
___

# 3) Does having 'manager' in your title effect the amount a customer orders?


### 3-1 Inital Analysis
The customers that we do business with, there are different tiers of titles that they use. I want to know if a manager typically orders more than other customers.

$H_{o} =$ Having manager in your title, the amount a customer orders doesn't change  
$H_{a} =$ Having manager in your title, the amount a customer orders changes  
$\alpha = 0.05$

In [ ]:
get_column_info('Customer')

In [ ]:
q = '''SELECT c.CompanyName, c.ContactTitle, COUNT(*) AS orders_placed \
FROM OrderDetail od \
LEFT JOIN [Order] o ON od.OrderId = o.Id \
LEFT JOIN Customer c ON c.Id = o.CustomerId \
GROUP BY c.CompanyName'''

df = pd.read_sql_query(q, engine)
df.head(10)

In [ ]:
df.orders_placed.sum()

Good, we have all of the orders placed. There seems be a few ghost orders, someone placing some personal orders, or some testing data. Nevertheless, I will need to remove them.

In [ ]:
df.drop(index=0, inplace=True)
df.describe()

Now encoding manager title vs no mananger title.

In [ ]:
df['is_mgmt'] = np.where(df.ContactTitle.str.contains('anager'),1,0) #also could have used .lower() to include the full word

In [ ]:
df.head(10)

In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='orders_placed',x='is_mgmt', data=df)
plt.title('Violin plots: comparing the distribution of management vs non-management')
plt.xlabel('Is management')
plt.ylabel('Number of items in order');

In [ ]:
df.groupby('is_mgmt').orders_placed.idxmax(),df.groupby('is_mgmt').orders_placed.max()

In [ ]:
df.drop(index=[66,19], inplace=True)
#df.drop(index=[57], inplace=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='orders_placed',x='is_mgmt', data=df)
plt.title('Violin plots: comparing the distribution of management vs non-management')
plt.xlabel('Is management')
plt.ylabel('Number of items in order');

## 3-2 Testing

I will perform both the independent T-test as well as Mann-Whitney U test.  

See section 2-2 for supporting documentation

In [ ]:
mgmt_yes = df.orders_placed[df.is_mgmt == 1]
mgmt_no = df.orders_placed[df.is_mgmt == 0]
ttest_ind(mgmt_yes, mgmt_no)

In [ ]:
mannwhitneyu(mgmt_no, mgmt_yes)

## 3-3 Results

- Failed to reject the null hypothesis. As a customer having manager in your title, the quantity ordered _doesn't signifgantly change_ compared to customers without that title  
- p-value > $\alpha$ :: 0.2241 > 0.05
- Further testing: change the criteria in which the customers are grouped. Include 'owner' title with the 'manager' title and compare results.
___
___

# 4) Is there a difference in quantity of products given their average shelf life?

## 4-1 Initial Analysis
Northwind Traders sell different types of products. I encode the types of products into perishable vs non-perishable. 

$H_{o} =$ The quantity of perishable products ordered is the same as the quantity of non-perishable products ordered  
$H_{\alpha} =$ The quantity of perishable products ordered is the different as the quantity of non-perishable products ordered  
$\alpha = 0.05$

In [ ]:
get_column_info('OrderDetail')

In [ ]:
q = '''SELECT od.ProductId AS prod_id, od.Quantity, od.UnitPrice, cat.CategoryName \
FROM OrderDetail od \
LEFT JOIN Product p ON od.ProductId = p.Id \
LEFT JOIN Category cat ON p.CategoryId = cat.Id \
'''

df = pd.read_sql_query(q,engine)
df.head()

In [ ]:
df.shape

All 2155 items in all the orders are included.

In [ ]:
df.CategoryName.unique()

Encoding each different category into perishable/non-perishable where 1 indicates perishable

In [ ]:
#goes_bad = ['Produce', 'Meat/Poultry', 'Seafood', 'Dairy Products']
#non_perish = ['Beverages', 'Condiments', 'Confections', 'Grains/Cereals']

def perishable(row):
    if row['CategoryName'] == 'Produce':
        return 1
    if row['CategoryName'] == 'Meat/Poultry':
        return 1
    if row['CategoryName'] == 'Seafood':
        return 1
    if row['CategoryName'] == 'Dairy Products':
        return 1
    if row['CategoryName'] == 'Beverages':
        return 0
    if row['CategoryName'] == 'Condiments':
        return 0
    if row['CategoryName'] == 'Confections':
        return 0
    if row['CategoryName'] == 'Grains/Cereals':
        return 0

In [ ]:
df['perish'] = df.apply(lambda row : perishable(row), axis=1)

In [ ]:
df.head()

Now to plot and compare each category

In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='Quantity',x='perish', data=df)
plt.title('Violin plots: comparing the distribution of perishable items')
plt.xlabel('Is perishable')
plt.ylabel('Quantity ordered')
#plt.savefig('vioin perish');

In [ ]:
df.groupby('perish')['Quantity'].hist()

## 4-2 Testing

Once again, I will perform both the independent T-test as well as Mann-Whitney U test.  

See section 2-2 for supporting documentation

In [ ]:
goes_bad = df.Quantity[df.perish == 1]
no_bad = df.Quantity[df.perish == 0]
mannwhitneyu(no_bad,goes_bad)

In [ ]:
ttest_ind(no_bad,goes_bad)

## 4-3 Results
- Failed to reject the null hypothesis. The quantity ordered of perishable items _doesn't significantly change_ The quantity ordered of non-perishable items 
- (p-value > $\alpha$ :: 0.3518 > 0.05)
- Further testing: Reach out to the data engineers and obtain 'good until' dates and use that as a metric
___
___
___
___

RESOUCES:

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3116565/

https://pythonfordatascience.org/